# Traffic data analysis

The objective of this notebook is to use data provided by Victor Couture and team to crown "Canada's Worst Commute". The output is a ranking of the routes that are worst in Canada, according to methodology that is vetted by experts.

In [38]:
import pandas as pd

Below are some assumptions that we've landed on based on speaking to experts.

In [39]:
# In kilometres.
MIN_TRIP_DISTANCE = 10

# Local hour when rush hour starts.
MORNING_RUSH_HOUR_START = 8
MORNING_RUSH_HOUR_END = 10
EVENING_RUSH_HOUR_START = 17
EVENING_RUSH_HOUR_END = 19

# Minimum number of data points for both rush hour and non-rush hour times.
MIN_DATA_POINTS_NONRUSHHOUR = 10
MIN_DATA_POINTS_RUSHHOUR = 5

# The start and end dates for the 2025 analysis.
# We decided to go back 12 months from Oct. 1, 2025
START_DATE = pd.Timestamp(f"2024-10-01")
END_DATE = pd.Timestamp(f"2025-09-30")

Now we read in the data.

This data has been converted from the original Stata format, which Pandas had some difficulty with. The conversion script is separate, but is fairly simple. The only filtering that was done then was to remove data earlier than Jan. 1, 2024 and to remove columns I know we don't need for this analysis. This was done to keep the file small.

In [40]:
raw = pd.read_parquet('data/victor/alltrips_canada_2024_2025.parquet')

# Create a datetime column for analysis
raw.head(1)

,tripid,mode,citycode,cityname_corrected,time_full_str,traffic_min,notraffic_min,dayofweek,tz,trip_dist,lat_dest,lon_dest,lat_orig,lon_orig
0,101497520,0.0,20370,Calgary,20240111 00:56:12,13.3,13.1,3,America/Edmonton,9.548,51.060974,-113.951439,51.04414,-114.059586


Going to create a copy dataframe here, because I will go back to this raw data at the end for some checks.

In [41]:
df = raw.copy()

We only have a few bits of cleaning to do. Our export method doesn't like é, so we'll change that to e.

There is also a small type in Kitchener-Cambridge-Waterloo that we can fix here.

In [42]:
df["cityname_corrected"] = df["cityname_corrected"].str.replace("é", "e")
df["cityname_corrected"] = df["cityname_corrected"].replace("Kitchener-Cambrigde-Waterloo", "Kitchener-Cambridge-Waterloo")

Now I'm going to create a new column to combine lat/lon for origin and dest. This just makes it easy for me to copy/paste coordinates over to google maps for presentation to the editorial team.

In [43]:
df['origin'] = df['lat_orig'].astype(str) + ',' + df['lon_orig'].astype(str)
df['dest'] = df['lat_dest'].astype(str) + ',' + df['lon_dest'].astype(str)

df = df.drop(columns=["lat_dest", "lon_dest", "lat_orig", "lon_orig"])

I will convert the time string into datetime.

Note that the data is already in local timezone, so no conversion is necessary here.

In [44]:
# Create a datetime column for analysis
df['time_dt'] = pd.to_datetime(df['time_full_str'])

df = df.drop(columns=["time_full_str"])

Now we can filter for only the data in the time frames we're interested in.

In [45]:
df = df[(df['time_dt'] <= END_DATE) & (df['time_dt'] >= START_DATE)]

Check data start time and end time.

In [46]:
display(df["time_dt"].min(), df["time_dt"].max())

Timestamp('2024-10-29 05:39:11')

Timestamp('2025-09-29 23:59:46')

What cities is there data for?

In [47]:
df.loc[:, ["cityname_corrected", "tz"]].drop_duplicates()

,cityname_corrected,tz
3970387,Calgary,America/Edmonton
3971133,Edmonton,America/Edmonton
3971843,Halifax,America/Halifax
3972826,Hamilton,America/Toronto
3973695,Kitchener-Cambridge-Waterloo,America/Toronto
3974686,London,America/Toronto
3975805,Montreal,America/Toronto
3977878,Oshawa,America/Toronto
3978757,Ottawa,America/Toronto
3980304,Quebec,America/Toronto


For this table, I’ve excluded routes that are less than 10km long, and only included routes with at least 3 data points collected over the past 6 months in both non-rush hour and rush hour (8-10am, 5-7pm) times. I've excluded everything under 10km, because that doesn't really feel like it qualifies as a commute.

In [48]:
# --- Rush Hour Analysis most recent year
weekdays_df = df[(df['time_dt'].dt.weekday < 5) & (df['trip_dist'] >= MIN_TRIP_DISTANCE) & (df['mode'] == 0)].copy()

hour = weekdays_df['time_dt'].dt.hour

is_rush_hour = ((hour >= MORNING_RUSH_HOUR_START) & (hour < MORNING_RUSH_HOUR_END)) | ((hour >= EVENING_RUSH_HOUR_START) & (hour < EVENING_RUSH_HOUR_END))

weekdays_df['period'] = 'Non-Rush Hour'
weekdays_df.loc[is_rush_hour, 'period'] = 'Rush Hour'

We'll start the analysis here by classifying each data point as a "rush hour" data point, or a "not rush hour" data point.

This is done according to the rush hour cutoffs from above.

Note we are using mean here, not median. This was an editorial decision, and experts have told us that mean and median are both acceptable to use here.

In [49]:
grouping_cols = ['tripid', 'cityname_corrected','period']
agg_times = (weekdays_df
             .groupby(grouping_cols)['traffic_min']
             .agg(['mean', 'count'])
             .reset_index()
             )

agg_times.head(3)

,tripid,cityname_corrected,period,mean,count
0,101497530,Calgary,Non-Rush Hour,24.066667,13
1,101497530,Calgary,Rush Hour,27.936111,6
2,101497531,Calgary,Non-Rush Hour,24.150000,12


Now we reshape a little to collect up all the data points for each tripid.

In [75]:
comparison_table = agg_times.pivot_table(
    index=["tripid", "cityname_corrected"],
    columns='period',
    values=['mean', 'count']
).reset_index()

comparison_table.head(20)

tripid cityname_corrected         count                    mean  \
period                               Non-Rush Hour Rush Hour Non-Rush Hour   
0       101497530            Calgary          13.0       6.0     24.066667   
1       101497531            Calgary          12.0       NaN     24.150000   
2       101497540            Calgary          10.0       6.0     19.196667   
3       101497541            Calgary          13.0       5.0     19.565385   
4       101497551            Calgary          14.0       5.0     13.625000   
5       101497560            Calgary           6.0       1.0     16.913889   
6       101497561            Calgary           9.0       3.0     18.037037   
7       101497570            Calgary          14.0       7.0     24.104762   
8       101497571            Calgary          10.0       3.0     24.615000   
9       101497590            Calgary          11.0       4.0     26.387879   
10      101497591            Calgary          16.0       5.0     25.651042   
11      101497600            Calgary          10.0       4.0     24.968333   
12      101497601            Calgary          13.0       5.0     21.792308   
13      101497620            Calgary          12.0       7.0     27.043056   
14      101497621            Calgary          11.0       2.0     26.839394   
15      101497640            Calgary           4.0       2.0     26.616667   
16      101497641            Calgary          19.0       2.0     23.810526   
17      101497650            Calgary          11.0       3.0     18.022727   
18      101497651            Calgary          10.0       2.0     19.461667   
19      101497660            Calgary           2.0       NaN     13.708333   

                   
period  Rush Hour  
0       27.936111  
1             NaN  
2       21.172222  
3       23.350000  
4       15.130000  
5       17.566667  
6       21.127778  
7       28.421429  
8       29.266667  
9       26.700000  
10      29.396667  
11      33.183333  
12      27.456667  
13      34.133333  
14      28.666667  
15      30.366667  
16      29.691667  
17      20.050000  
18      20.233333  
19            NaN

Now we just flatten the columns so they're a bit cleaner.

In [76]:
# Flatten multiindex column names
comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]

comparison_table.head(3)

,tripid_,cityname_corrected_,count_Non-Rush Hour,count_Rush Hour,mean_Non-Rush Hour,mean_Rush Hour
0,101497530,Calgary,13.0,6.0,24.066667,27.936111
1,101497531,Calgary,12.0,NaN,24.150000,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222


And rename them because they're a bit messy.

In [77]:
comparison_table.rename(columns={
    'mean_Non-Rush Hour': 'mean_non_rush_hour',
    'mean_Rush Hour': 'mean_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'trip_dist_': 'trip_dist',
}, inplace=True)

Now I want to know what the average travel time is for each trip with no traffic.

In [78]:
# Compute a single overall mean for notraffic_min (include all periods)
notraffic_overall = (weekdays_df
                     .groupby(['tripid', 'cityname_corrected'])['notraffic_min']
                     .mean()
                     .reset_index()
                     .rename(columns={'notraffic_min': 'mean_notraffic'})
                     )

notraffic_overall.head(3)

,tripid,cityname_corrected,mean_notraffic
0,101497530,Calgary,22.245614
1,101497531,Calgary,21.687500
2,101497540,Calgary,17.211458


And because it's of interest to the story team, I'd like to include a column that shows the 90th percentile time for each trip.

In [79]:
# Compute 90th percentile travel time during Rush Hour per trip (p90)
rush_df = weekdays_df[weekdays_df['period'] == 'Rush Hour']
p90_rush = (rush_df
            .groupby(['tripid', 'cityname_corrected'])['traffic_min']
            .quantile(0.9)
            .reset_index()
            .rename(columns={'traffic_min': 'p90_rush_hour'})
            )

p90_rush.head(3)

,tripid,cityname_corrected,p90_rush_hour
0,101497530,Calgary,32.450000
1,101497540,Calgary,24.583333
2,101497541,Calgary,26.476667


And I'd also like to know roughly how long each trip is. Note that each trip appears to be a slightly different distance, even within the same route ideas. This is because Google Maps may take slightly different routes each time.

In [80]:
# Compute a single overall mean for notraffic_min (include all periods)
tripdist = (weekdays_df
                     .groupby(['tripid', 'cityname_corrected'])['trip_dist']
                     .mean()
                     .reset_index()
                     .rename(columns={'trip_dist': 'mean_trip_dist'})
                     )

tripdist.head(3)

,tripid,cityname_corrected,mean_trip_dist
0,101497530,Calgary,18.731105
1,101497531,Calgary,20.683417
2,101497540,Calgary,12.493187


Now we merge the overall notraffic mean, p90, and tripdist tables into the comparison table

In [81]:
comparison_table = comparison_table.merge(notraffic_overall, on=['tripid', 'cityname_corrected'], how='left')
comparison_table = comparison_table.merge(p90_rush, on=['tripid', 'cityname_corrected'], how='left')
comparison_table = comparison_table.merge(tripdist, on=['tripid', 'cityname_corrected'], how='left')

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187


Origin and destination should be the exact same for each trip id. But in case they differ slightly, we'll just grab the first of each pulled for each route id and add to our table.

In [82]:
orig_dest = weekdays_df.groupby(['tripid', 'cityname_corrected']).agg({'origin': 'first', 'dest': 'first'}).reset_index()
comparison_table = comparison_table.merge(orig_dest, on=['tripid', 'cityname_corrected'], how='left')

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575"
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346"
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335"


Now we'll calculate some columns. We start with the difference in travel time between freeflow (no traffic) and rush hour.

In [83]:
comparison_table['rush_vs_freeflow_diff'] = comparison_table['mean_rush_hour'] - comparison_table['mean_notraffic']

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764


Now rush hour time versus freeflow PERCENT.

This is the percent increase in travel time, on average, during rush hour versus free flow.

In [84]:
comparison_table['rush_vs_freeflow_pct'] = comparison_table.apply(lambda r: (r['rush_vs_freeflow_diff'] / r['mean_notraffic'] * 100) if r['mean_notraffic'] else 0, axis=1)

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497,25.580310
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764,23.012367


Now finall we compare the 90th percentile travel time to freeflow by subtraction.

In [85]:
comparison_table['p90_vs_freeflow'] = comparison_table['p90_rush_hour'] - comparison_table['mean_notraffic']

comparison_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_vs_freeflow
0,101497530,Calgary,13.0,6.0,24.066667,27.936111,22.245614,32.450000,18.731105,"51.04198,-114.051346","51.12566,-114.22575",5.690497,25.580310,10.204386
1,101497531,Calgary,12.0,NaN,24.150000,NaN,21.687500,NaN,20.683417,"51.12566,-114.22575","51.04198,-114.051346",NaN,NaN,NaN
2,101497540,Calgary,10.0,6.0,19.196667,21.172222,17.211458,24.583333,12.493187,"51.044357,-114.061646","50.949066,-114.085335",3.960764,23.012367,7.371875


If we have no data points in rush hour, let's just drop that trip.

In [86]:
comparison_table = comparison_table[~comparison_table['count_rush_hour'].isna()]

In [87]:
# Convert count columns to integers where present
for col in ['count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

Now we drop anything without enough data points, defined our assumptions constants above.

In [88]:
print(f"Before dropping min data points: {len(comparison_table)}.")
comparison_table = comparison_table[(comparison_table['count_rush_hour'] >= MIN_DATA_POINTS_RUSHHOUR)]
print(f"After dropping min data points: {len(comparison_table)}.")

Before dropping min data points: 58119.
After dropping min data points: 17235.


Now make sure count columns are integers.

In [89]:
# These columns should be floats of 1 decimal.
comparison_table['mean_trip_dist'] = comparison_table['mean_trip_dist'].round(1)

# These columns should be floats of 2 decimals.
num_cols = ['mean_non_rush_hour', 'mean_rush_hour', 'mean_notraffic', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour']

for c in num_cols:
    if c in comparison_table.columns:
        comparison_table[c] = comparison_table[c].round(2)

In [91]:
# Final sorting and filtering
final_table = comparison_table.sort_values('rush_vs_freeflow_diff', ascending=False)

final_table.head(3)

,tripid,cityname_corrected,count_non_rush_hour,count_rush_hour,mean_non_rush_hour,mean_rush_hour,mean_notraffic,p90_rush_hour,mean_trip_dist,origin,dest,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_vs_freeflow
14009,101622821,Toronto,11.0,5,43.82,73.67,33.33,77.40,39.9,"43.45908,-79.66448","43.653397,-79.38296",40.34,121.02,44.066667
12635,101615481,Toronto,14.0,5,44.63,71.52,31.51,75.85,43.1,"43.418945,-79.71838","43.652653,-79.38433",40.01,126.97,44.344561
14958,101627951,Toronto,18.0,5,43.91,71.71,34.86,85.24,51.2,"43.35732,-79.798035","43.64694,-79.38845",36.85,105.72,50.381304


Going through and checking each route, I noticed one route that I don't think we should include, so I've set up a list of exclusions that I'll filter our list on now.

In [92]:
exclusions = [
    "303934731", # This is a route in Montreal that loops back on itself on Google Maps. I think it's likely a much shorter route.
]

final_table = final_table[~final_table["tripid"].isin(exclusions)]

Add a column for rank, to make it easier to understand for the story team.

In [93]:
final_table["rank"] = range(1,len(final_table)+1)

Now let's take a look at the final table, with rearranged column order in way that makes more sense.

In [96]:
display_cols = ["rank", 'tripid','cityname_corrected', 'origin', 'dest', 'mean_trip_dist', 'mean_notraffic', 'mean_rush_hour', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour', "p90_vs_freeflow"]

In [97]:
final_table.reset_index().set_index("rank")[[col for col in display_cols if col != "rank"]].head(20)

,tripid,cityname_corrected,origin,dest,mean_trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_rush_hour,p90_vs_freeflow
rank,,,,,,,,,,,
1,101622821,Toronto,"43.45908,-79.66448","43.653397,-79.38296",39.9,33.33,73.67,40.34,121.02,77.40,44.066667
2,101615481,Toronto,"43.418945,-79.71838","43.652653,-79.38433",43.1,31.51,71.52,40.01,126.97,75.85,44.344561
3,101627951,Toronto,"43.35732,-79.798035","43.64694,-79.38845",51.2,34.86,71.71,36.85,105.72,85.24,50.381304
4,101612671,Toronto,"43.48599,-79.686455","43.658115,-79.38193",35.9,32.08,68.84,36.76,114.59,76.82,44.734615
5,101622220,Toronto,"43.6529,-79.383644","43.72911,-79.75203",43.7,33.57,69.54,35.97,107.15,79.85,46.280208
6,101621581,Toronto,"43.457336,-79.6583","43.655632,-79.390854",40.6,37.11,72.86,35.75,96.33,78.44,41.332500
7,101608211,Toronto,"43.461823,-79.691605","43.654392,-79.382614",39.3,32.80,68.42,35.61,108.57,75.72,42.914583
8,101627971,Toronto,"43.37529,-79.82035","43.65489,-79.37266",56.7,42.21,77.60,35.39,83.84,87.62,45.414167
9,101619411,Toronto,"43.495205,-79.69126","43.65315,-79.3833",35.1,29.10,63.65,34.56,118.76,79.01,49.909231


Find the longest travel time for our gold medal.

In [101]:
df[df["tripid"] == "101622821"].sort_values("traffic_min", ascending=False).head(3)

,tripid,mode,citycode,cityname_corrected,traffic_min,notraffic_min,dayofweek,tz,trip_dist,origin,dest,time_dt
6356317,101622821,0.0,20402,Toronto,78.8,29.950000,2,America/Toronto,39.732,"43.45908,-79.66448","43.653397,-79.38296",2025-05-14 08:07:02
6356323,101622821,0.0,20402,Toronto,78.6,31.250000,5,America/Toronto,39.731,"43.45908,-79.66448","43.653397,-79.38296",2025-07-05 17:21:26
6356319,101622821,0.0,20402,Toronto,75.3,37.316667,2,America/Toronto,39.555,"43.45908,-79.66448","43.653397,-79.38296",2025-05-21 17:50:16


In [107]:
frames = []

for city in final_table["cityname_corrected"].unique():
    
    city_filter_final_table = final_table.reset_index().set_index("rank")[[col for col in display_cols if col != "rank"]]
    
    top_result_in_city = city_filter_final_table[city_filter_final_table["cityname_corrected"] == city].head(1)
    
    frames.append(top_result_in_city)
    
city_winners = pd.concat(frames)

city_winners.reset_index().to_csv("city_winners.csv")

display(city_winners)

,tripid,cityname_corrected,origin,dest,mean_trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_rush_hour,p90_vs_freeflow
rank,,,,,,,,,,,
1,101622821,Toronto,"43.45908,-79.66448","43.653397,-79.38296",39.9,33.33,73.67,40.34,121.02,77.40,44.066667
23,101634500,Vancouver,"49.254524,-123.11747","49.13831,-122.82736",38.4,39.87,71.71,31.84,79.87,78.23,38.363636
43,303950241,Montreal,"45.751167,-73.598656","45.427303,-73.618225",53.9,45.26,75.58,30.33,67.01,83.33,38.069524
1212,5273331,Oshawa,"43.628483,-79.39596","43.878773,-78.9387",60.1,63.18,80.95,17.77,28.12,102.57,39.388750
1498,14162550,Calgary,"51.03615,-114.05375","51.163784,-114.1207",28.4,27.71,44.33,16.62,59.98,46.56,18.855000
1503,13248751,Ottawa,"45.349422,-75.76914","45.422485,-75.693954",13.3,14.69,31.28,16.59,112.97,36.77,22.077451
1530,14165270,Edmonton,"53.543312,-113.5034","53.4107,-113.53224",18.1,26.05,42.54,16.49,63.32,48.14,22.094444
1627,5275061,St. Catharines-Niagara,"43.628483,-79.39596","43.20611,-79.2137",114.5,94.62,110.77,16.16,17.08,118.39,23.770926
2057,304026560,Quebec,"46.74544,-71.229385","46.829834,-71.242775",24.3,18.76,33.46,14.69,78.30,39.18,20.419167


In [102]:
(final_table
 .reset_index()
 .set_index("rank")[[col for col in display_cols if col != "rank"]]
 .to_csv("results-victor.csv")
 )

## Mapping results

This code is used to print out start and end points for each path. I'm not really using it for analysis, more for communicating results to the story team. It does not need to be reviewed.

In [99]:
# # Save top-3 route maps per city and fetch fresh geometry from Google Directions for each origin/destination
# import os
# import pathlib
# import re
# import time
# import requests
# import polyline
# import matplotlib.pyplot as plt
# import geopandas as gpd
# from shapely.geometry import LineString

# out_dir = pathlib.Path('plots_by_route')
# out_dir.mkdir(parents=True, exist_ok=True)

# # Read API key (file fallback then env)
# API_KEY = None
# key_path = os.path.join(os.getcwd(), 'google_maps_api_key.txt')
# if os.path.exists(key_path):
#     with open(key_path, 'r', encoding='utf-8') as fh:
#         API_KEY = fh.read().strip()
# if not API_KEY:
#     API_KEY = os.getenv('GOOGLE_API_KEY')
# if not API_KEY:
#     raise RuntimeError('Google Maps API key not found. Add google_maps_api_key.txt or set GOOGLE_API_KEY')

# def safe(s):
#     if s is None:
#         return 'unknown'
#     s = str(s)
#     s = re.sub(r'[^A-Za-z0-9_\-]', '_', s)
#     return s[:180]

# if 'final_table' not in globals():
#     raise RuntimeError('`final_table` not found. Run the Rush Hour Analysis cell first.')

# ft = final_table.reset_index()
# if 'rank' not in ft.columns:
#     ft.insert(0, 'rank', range(1, len(ft) + 1))

# # cache to avoid duplicate API calls for identical origin/dest within this run
# route_cache = {}

# def fetch_route_geometry(origin_str, dest_str, max_retries=3):
#     key = f'{origin_str}|{dest_str}'
#     if key in route_cache:
#         return route_cache[key]
#     url = 'https://maps.googleapis.com/maps/api/directions/json'
#     params = {
#         'origin': origin_str,
#         'destination': dest_str,
#         'alternatives': 'false',
#         'mode': 'driving',
#         'key': API_KEY
#     }
#     backoff = 1.0
#     for attempt in range(1, max_retries + 1):
#         try:
#             resp = requests.get(url, params=params, timeout=15)
#             resp.raise_for_status()
#             j = resp.json()
#         except Exception as e:
#             if attempt == max_retries:
#                 raise
#             time.sleep(backoff)
#             backoff *= 2
#             continue
#         status = j.get('status')
#         if status != 'OK' or not j.get('routes'):
#             # For non-retryable API errors, return None
#             return None
#         ov = j['routes'][0].get('overview_polyline', {}).get('points')
#         if not ov:
#             return None
#         coords_latlon = polyline.decode(ov)  # list of (lat, lon)
#         coords = [(lon, lat) for lat, lon in coords_latlon]
#         ls = LineString(coords)
#         # create polygon buffer of 100 m by projecting to web mercator
#         single_line_gdf = gpd.GeoDataFrame([{'tripid': None, 'origin': origin_str, 'dest': dest_str, 'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#         web = single_line_gdf.to_crs(epsg=3857)
#         web['polygon_100m'] = web.geometry.buffer(100)
#         poly = gpd.GeoDataFrame(web.copy(), geometry=web['polygon_100m'], crs=web.crs).to_crs(epsg=4326)
#         route_cache[key] = (ls, poly)
#         return ls, poly
#     return None

# for city in ft['cityname_corrected'].unique():
#     city_rows = ft[ft['cityname_corrected'] == city].sort_values('rush_vs_freeflow_diff', ascending=False).head(3)
#     if city_rows.empty:
#         continue
#     for _, crow in city_rows.iterrows():
#         rank = int(crow.get('rank', -1))
#         tripid = crow.get('tripid')
#         origin_val = crow.get('origin')
#         dest_val = crow.get('dest')
#         if not origin_val or not dest_val:
#             print(f'Missing origin/dest for trip {tripid} (rank {rank}) in {city}; skipping')
#             continue
#         # origin_val/dest_val should be 'lat,lon' strings; ensure trimmed
#         origin_str = str(origin_val).strip()
#         dest_str = str(dest_val).strip()
#         try:
#             res = fetch_route_geometry(origin_str, dest_str)
#         except Exception as e:
#             print(f'Error fetching route for {tripid} in {city}: {e}; skipping')
#             continue
#         if not res:
#             print(f'No route geometry returned for {tripid} in {city}; skipping')
#             continue
#         ls, poly = res
#         fname = out_dir / f"{rank:03d}_{safe(tripid)}_{safe(city)}.png"
#         fig, ax = plt.subplots(1, 1, figsize=(8, 6))
#         try:
#             if poly is not None and not poly.empty:
#                 poly.to_crs(epsg=3857).plot(ax=ax, color='orange', alpha=0.4, edgecolor='darkorange')
#             single_line = gpd.GeoDataFrame([{'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#             single_line.to_crs(epsg=3857).plot(ax=ax, color='red', linewidth=3)
#             minx, miny, maxx, maxy = single_line.to_crs(epsg=3857).total_bounds
#             buf = max((maxx - minx), (maxy - miny)) * 0.25 if maxx > minx and maxy > miny else 200
#             ax.set_xlim(minx - buf, maxx + buf)
#             ax.set_ylim(miny - buf, maxy + buf)
#             try:
#                 import contextily as ctx
#                 ctx.add_basemap(ax)
#             except Exception:
#                 pass
#             ax.set_axis_off()
#             ax.set_title(f'Rank {rank} — Trip {tripid} — {city}')
#             fig.savefig(fname, dpi=150, bbox_inches='tight')
#             plt.close(fig)
#             print(f'Saved {fname}')
#         except Exception as e:
#             print(f'Failed to save route {tripid} (rank {rank}) for city {city}: {e}')
